# Marketing and Retail Analytics

### Problem Statement

OList is an e-commerce company that has faced some losses recently and they want to manage their inventory very well so as to reduce any unnecessary costs that they might be bearing. In this assignment, you have to manage the inventory cost of this e-commerce company OList. You need to identify top products that contribute to the revenue and also use market basket analysis to analyse the purchase behaviour of individual customers to estimate with relative certainty, what items are more likely to be purchased individually or in combination with some other products.

In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

### Data cleaning

Exploring and cleaning the datasets one by one from the excel file.

#### 1. Orders

In [ ]:
#Reading the orders dataset
orders = pd.read_excel("Retail_data.xlsx", sheet_name = "orders")
orders.head()

In [ ]:
orders.shape

In [ ]:
#Checking duplicate values for the column order id
orders["order_id"].duplicated().sum()

In [ ]:
#checking duplicate values for the column customer id
orders["customer_id"].duplicated().sum()

In [ ]:
orders["customer_id"].value_counts().sort_values(ascending=False).head(10)

In [ ]:
orders.head()

In [ ]:
orders["order_status"].value_counts()

In [ ]:
#Filtering the order status to delivered
orders = orders[orders["order_status"] == "delivered"]
orders.shape

In [ ]:
orders.head()

In [ ]:
#Checking null values present in the data set
orders.isnull().sum()

In [ ]:
#Filling the missing values
orders["order_approved_at"].fillna(orders["order_purchase_timestamp"],inplace=True)
orders["order_delivered_timestamp"].fillna(orders["order_estimated_delivery_date"],inplace=True)

In [ ]:
orders.isnull().sum()

#### 2. Order_items

In [ ]:
#Reading order items dataset
order_items = pd.read_excel("Retail_data.xlsx",sheet_name="order_items")
order_items.head()

In [ ]:
order_items.shape

In [ ]:
#Checking duplicates
order_items.duplicated().sum()

In [ ]:
#Checking null values
order_items.isnull().sum()

#### 3. Customers

In [ ]:
#Reading customers dataset
customers = pd.read_excel("Retail_data.xlsx",sheet_name="customers")
customers.head()

In [ ]:
customers.shape

In [ ]:
#Checking null values
customers.isnull().sum()

In [ ]:
#Checking duplicates
customers["customer_id"].duplicated().sum()

In [ ]:
#Dropping the duplicates
customers.drop_duplicates(subset="customer_id", keep="first", inplace=True)

In [ ]:
customers.shape

#### 4. Payments

In [ ]:
#Reading payments dataset
payments = pd.read_excel("Retail_data.xlsx",sheet_name="payments")
payments.head()

In [ ]:
payments.shape

In [ ]:
#Checking duplicates
payments.duplicated().sum()

In [ ]:
#Checking null values
payments.isnull().sum()

In [ ]:
#Checking the most used 'payment_type' and their counts.
payments["payment_type"].value_counts()

In [ ]:
#Since there are only 3 recors affected, we can drop these records.

i=payments[payments['payment_type']=='not_defined'].index
payments.drop(i, axis=0, inplace=True)

In [ ]:
payments["payment_type"].value_counts()

#### 5. Products

In [ ]:
#Reading products dataset
products = pd.read_excel("Retail_data.xlsx",sheet_name="products")
products.head()

In [ ]:
products.shape

In [ ]:
#Checking duplicates
products.duplicated().sum()

In [ ]:
#checking null values
products.isnull().sum()

In [ ]:
#Replacing the null values with mode value
products["product_category_name"].mode()

In [ ]:
products["product_category_name"].fillna(products["product_category_name"].mode()[0],inplace=True)

In [ ]:
products.isnull().sum()

In [ ]:
products.describe()

In [ ]:
#Visualizing numerical columns to fill the missing values
num_col =["product_weight_g","product_length_cm","product_height_cm","product_width_cm"]

In [ ]:
for i in num_col:
    plt.figure(figsize=(12,5))
    sns.distplot(products[i])
    plt.axvline(products[i].mean(), color="blue")
    plt.axvline(products[i].median(), color="red")
    plt.show()

In [ ]:
for i in num_col:
    products[i].fillna(products[i].median(), inplace=True)

In [ ]:
products.isnull().sum()

### Merging Dataframes

Creating a new dataframe 'New_retail', which will contain the data merged using different dataframes.

In [ ]:
#Merging 'orders' with 'order_items'
New_retail = pd.merge(orders,order_items,how='inner',on='order_id')

In [ ]:
New_retail.shape

In [ ]:
#Merging 'New_retail' with 'products'
New_retail = pd.merge(New_retail,products,how='inner',on='product_id')

In [ ]:
New_retail.shape

In [ ]:
#Merging 'New_retail' with 'payments'
New_retail = pd.merge(New_retail,payments,how='inner',on='order_id')

In [ ]:
New_retail.shape

In [ ]:
#Merging 'New_retail' with 'customers'
New_retail = pd.merge(New_retail,customers,how='inner',on='customer_id')

In [ ]:
New_retail.shape

In [ ]:
New_retail.head()

In [ ]:
New_retail.isnull().sum()

In [ ]:
#Checking for outliers
plt.figure(figsize=(10,6))
sns.boxplot(data=New_retail, y=New_retail['price'])
plt.figure(figsize=(10,6))
sns.boxplot(data=New_retail, y=New_retail['shipping_charges'])
plt.show()

In [ ]:
#Visualizing price and shipping charges for imputation
plt.figure(figsize=(15,5))
sns.distplot(New_retail.price)
plt.axvline(New_retail.price.mean(), color="red")
plt.axvline(New_retail.price.median(), color="blue")
plt.figure(figsize=(15,4))
sns.distplot(New_retail.shipping_charges)
plt.axvline(New_retail.shipping_charges.mean(), color="red")
plt.axvline(New_retail.shipping_charges.median(), color="blue")
plt.show()

In [ ]:
#Imputing outliers using median values
New_retail.loc[((New_retail['price']>3000) & (New_retail['product_category_name']=='toys')), 'price']=New_retail['price'].median()
New_retail.loc[New_retail['shipping_charges']>100, 'shipping_charges'] = New_retail['shipping_charges'].median()

### Exporting cleaned data to excel

In [ ]:
from pandas import ExcelWriter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Extracting the clean datasheets to be uploaded
with pd.ExcelWriter(r"C:\Users\Dell\Downloads\UpGrad\Capstone Project\Cleaned_Retail_Data.xlsx") as excel_sheets:
    #Extracting the clean datasheets
    orders.to_excel(excel_sheets, sheet_name="orders", index=False)
    order_items.to_excel(excel_sheets, sheet_name="order_items", index=False)
    products.to_excel(excel_sheets, sheet_name="products", index=False)
    customers.to_excel(excel_sheets, sheet_name="customers", index=False)
    payments.to_excel(excel_sheets, sheet_name="payments", index=False)

# Market Basket Analysis

### Apriori Algorithm & Association Rule
Apriori algorithms is a data mining algorithm used for mining frequent itemsets and relevant association rules. It is devised to operate on a database that contain transactions -like, items bought by a customer in a store.

An itemset can be considered frequent if it meets a user-specified support threshold. For example, if the support threshold is set to 0.5(50%), a frequent itemset is a set of items that are bought/purchased together in atleast 50% of all transactions.

Association rules are a set of rules derived from a database, that can help determining relationship among variables in a large transactional database

In [ ]:
#Installing external package
!pip install mlxtend

In [ ]:
# Importing libraries for performing Market Basket Analysis
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
#Updating New_retail dataframe with only the required columns for market basket analysis
New_retail = New_retail[['order_id','product_category_name', 'order_item_id']]

In [ ]:
#Checking for duplicate values
New_retail.duplicated().sum()

In [ ]:
#Dropping the duplicate values
New_retail.drop_duplicates(keep='first', inplace=True)

In [ ]:
#Creating prd_combo dataframe using pandas pivot table
product_combo = pd.pivot_table(data=New_retail,index='order_id',columns='product_category_name',
                              values='order_item_id',fill_value=0)

In [ ]:
product_combo.head()

In [ ]:
#For basket analysis encoding the data to 1s and 0s 
def encode_data(x):
    if x<=0:
        return 0
    if x>=1:
        return 1
product_combo_encode = product_combo.applymap(encode_data)

In [ ]:
product_combo_encode.shape

In [ ]:
#Dropping the Product_cataegories (columns) whose sum value (total_sale) is less than equal to 5
for column in product_combo_encode.columns:
    if (product_combo_encode[column].sum(axis=0, skipna=True)<=5):
        product_combo_encode.drop(column, inplace=True, axis=1)

In [ ]:
product_combo_encode.shape

Selecting only those order_ids where at least two items were purchased to find product combinations.This is required else the 'Toys' product_category will affect the whole analysis as the Support value for 'Toys' is biased due to its too much presence as single item orders 

In [ ]:
product_combo_encode = product_combo_encode[(product_combo_encode>0).sum(axis=1)>=2]
product_combo_encode.head()

#### Generating frequent itemsets from a list of items

First step in generation of association rules is to get all the frequent itemsets.Frequent itemsets are the ones which occur at least a minimum number of times in the transactions.

#### Support
This says how popular an itemset is, as measured by the proportion of transactions in which an itemset appears

If you discover that sales of items beyond a certain proportion tend to have a significant impact on your profits, you might consider using that proportion as your support threshold. You may then identify itemsets with support values above this threshold as significant itemsets
#### Confidence
This says how likely item Y is purchased when item X is purchased, expressed as {X -> Y}. This is measured by the proportion of transactions with item X, in which item Y also appears

One drawback of the confidence measure is that it might misrepresent the importance of an association. This is because it only accounts for how popular apples are, but not beers. If beers are also very popular in general, there will be a higher chance that a transaction containing apples will also contain beers, thus inflating the confidence measure. To account for the base popularity of both constituent items, we use a third measure called lift.
#### Confidence(x-->Y) = Support(X,Y)/Support(X)
#### Lift
This says how likely item Y is purchased when item X is purchased, while controlling for how popular item Y is. In Table 1, the lift of {apple -> beer} is 1,which implies no association between items. A lift value greater than 1 means that item Y is likely to be bought if item X is bought, while a value less than 1 means that item Y is unlikely to be bought if item X is bought
#### Lift (X-->Y) = Support(X,Y)/Support(X)*Support(Y)

In [ ]:
#Call apriori function and passing minimum support here we are passing 3%, which means at least 3% in total number of transaction the item should be present.'''

frequent_items = apriori(product_combo_encode, min_support=0.03, use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

frequent_items

#### Generating all possible rules from the frequent itemsets.

After the frequent itemsets are generated, identifying rules such as Confidence and Lift

In [ ]:
#We would apply association rules on frequent itemset to find product combinations
rules_conf = association_rules(frequent_items, metric="confidence", min_threshold=0.1)
rules_conf

In [ ]:
#Keeping minimum lift as >1.
rules_lift=rules_conf[(rules_conf['lift'] > 1)]
rules_lift

## Observations

Top products categories in groups of two’s are:

1. Toys and Bed Bath Table
2. Toys and Fashion Bags Accessories
3. Toys and Auto
4. Toys and Watches Gift


In [ ]:
# Filter the Basket with the sum of values having >=3

product_combo_encode_1 = product_combo_encode[(product_combo_encode > 0).sum(axis=1) >=3]
product_combo_encode_1

In [ ]:
frequent_items_1 = apriori(product_combo_encode_1, min_support=0.03, 
                            use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)
frequent_items_1

In [ ]:
#We would apply association rules on frequent itemset to find product combinations
rules_conf = association_rules(frequent_items_1, metric="confidence", min_threshold=0.1)
rules_conf

In [ ]:
#Keeping minimum lift as >1.
rules_lift=rules_conf[(rules_conf['lift'] > 1)]
rules_lift.head()

## Observations

Top products categories in groups of threes are:

1. Toys, Bed bath table and Housewares
2. Toys, Bed bath table and Office furniture	
3. Toys, Garden Tools and Computer Accessories
4. Toys, Furniture Decor and Electronics
5. Toys, Bed bath table and Health and Beauty